## Setup and imports
In this section we shall install the various dependencies that are required to run the analyzer

In [ ]:
%%capture
%pip install matplotlib
%pip install numpy
%pip install "jupyterlab>=3" "ipywidgets>=7.6"
%pip install plotly==5.15.0

In [ ]:
import json
import matplotlib.pyplot as plt
from pprint import pprint
from decimal import Decimal
from operator import truediv
import numpy
import statistics

## Reading the JSON Logs

In [ ]:
## Displaying Loaded Metadata
def showMetaDataLog(jsonLog):
    try:
        print("Compression Run for ",jsonLog["Compressor"])
        print("Run on ",jsonLog["Date"])
        print("Raw Event File ",jsonLog["Filename"])
        print("Events Compressed in Run ",jsonLog["eventCount"])
        print("Loaded Compressor Config \n",jsonLog["compressorConfig"])
    except Exception as e:
        print(e)

# - These Reports are to be shown for all the files

### Compression File

In [ ]:
compressionLogFile = open("../logTest.json")
compression_log = json.load(compressionLogFile)
showMetaDataLog(compression_log)

### Decompression File

In [ ]:
decompressionLogFile = open("../logTest_decompression.json")
decompression_log = json.load(decompressionLogFile)
showMetaDataLog(decompression_log)

## Analysis

### Compression Run

In [ ]:
eventList = compression_log["evdata"] # The event List
# No We design a filter dictionary to take up events based on filters
# Leave Empty if All Events are to be considered
eventListAnalysis = []
filter_dict = {
    #  "trig": "2"
}
if len(filter_dict.keys()) != 0:
    print("Filtering Events based on")
    pprint(filter_dict)
    for filter_field in filter_dict.keys():
        for event in eventList:
            if event["eventHeader"][filter_field] == filter_dict[filter_field]:
                eventListAnalysis.append(event)
else:
    print("No Filter given - Analysis for all events")
    eventListAnalysis = eventList
# Populate Metrics and Show preliminary results
compressionRatioList = [Decimal(event["compressionRatio"]) for event in eventListAnalysis]
timeTakenList = [Decimal(event["timeTaken"]) for event in eventListAnalysis]
inputSizeList = [Decimal(event["inputSize"]) for event in eventListAnalysis]
outputSizeList = [Decimal(event["outputSize"]) for event in eventListAnalysis]

compressionSpeedList  = list(map(truediv,inputSizeList,timeTakenList))
avaerageCompressionRatio = numpy.average(compressionRatioList)
avaerageCompressionSpeed = numpy.average(compressionSpeedList)
averageCompressionTime = numpy.average(timeTakenList)
x_axis = range(len(eventListAnalysis))


In [ ]:
## Plotiing Metrics
plt.style.use('classic')
fig, axs = plt.subplots(2,sharex=True)
fig.suptitle("Metrics Display")
axs[0].set_title("Compression Speed in bytes per microsecond")
axs[0].plot(x_axis,compressionSpeedList,'tab:orange')
axs[1].set_title("Compressin Ratio")
axs[1].plot(x_axis,compressionRatioList,'tab:purple')
print("Total Events:  ",len(eventList))
print("Total Events for Analysis:   ",len(eventListAnalysis))
print("The Average Compression Ratio achived is:  ",avaerageCompressionRatio)
print("The Average Compression Speed achived is:  ",avaerageCompressionSpeed)
print("The Average Compression Time achived is:  ",averageCompressionTime)

### Decompression Run

In [ ]:
eventList = decompression_log["evdata"] # The event List
# No We design a filter dictionary to take up events based on filters
# Leave Empty if All Events are to be considered
eventListAnalysis = []
filter_dict = {
    #  "trig": "2"
}
if len(filter_dict.keys()) != 0:
    print("Filtering Events based on")
    pprint(filter_dict)
    for filter_field in filter_dict.keys():
        for event in eventList:
            if event["eventHeader"][filter_field] == filter_dict[filter_field]:
                eventListAnalysis.append(event)
else:
    print("No Filter given - Analysis for all events")
    eventListAnalysis = eventList
# Populate Metrics and Show preliminary results
timeTakenList = [Decimal(event["timeTaken"]) for event in eventListAnalysis]
inputSizeList = [Decimal(event["inputSize"]) for event in eventListAnalysis]
outputSizeList = [Decimal(event["outputSize"]) for event in eventListAnalysis]

decompressionSpeedList  = list(map(truediv,outputSizeList,timeTakenList))

avaerageDecompressionSpeed = numpy.average(decompressionSpeedList)
averageDecompressionTime = numpy.average(timeTakenList)
x_axis = range(len(eventListAnalysis))


In [ ]:
## Plotiing Metrics
plt.style.use('classic')
plt.title("Decompression Speed in bytes per microsecond")
plt.plot(x_axis,decompressionSpeedList,'tab:orange')
print("Total Events:  ",len(eventList))
print("Total Events for Analysis:   ",len(eventListAnalysis))
print("The Average Compression Speed achived is:  ",avaerageDecompressionSpeed)
print("The Average Compression Time achived is:  ",averageDecompressionTime)

## Single Compressor Ananlysis

In [ ]:
Compressor = "ZSTD Compressor"
logsFolder = "/home/osboxes/gsocContributions/faser-common/EventFormats/apps/Logs"
# FIXME WE ASSUME THAT ALL RUNS ARE ON IDENTICAL SET OF EVENTS


### Compression Speed And Compression Ratio - Non Interactive

##### Compression Speed

In [ ]:
#Get a list of all the JSON files in the folder
import glob
jsonFiles = glob.glob(logsFolder + '/*.json')
# Get all the json files with the Compressor Key set to the target
targetFiles = []
for jsonfile in jsonFiles:
    with open(jsonfile) as f:
        jsonParse = json.load(f)
        if jsonParse["Compressor"]==Compressor:
            print (f'{f.name} ---> {jsonParse["Compressor"]}')
            targetFiles.append(jsonfile)
## Plotiing Metrics
plt.style.use('classic')
#figcompressor, axscompressor = plt.subplots(2,sharex=True)
figcompressor, axscompressor = plt.subplots()
figcompressor.suptitle("Metrics Display")
axscompressor.set_title("Compression Speed in bytes per microsecond")
#axscompressor[0].plot(x_axis,compressionSpeedList,'tab:orange')
# axscompressor[1].set_title("Compressin Ratio")
#axscompressor[1].plot(x_axis,compressionRatioList,'tab:purple')
for jsonFile in targetFiles:
    with open(jsonFile) as f:
        compression_log = json.load(f)
        eventList = compression_log["evdata"] # The event List
        # No We design a filter dictionary to take up events based on filters
        # Leave Empty if All Events are to be considered
        eventListAnalysis = []
        filter_dict = {
            #  "trig": "2"
        }
        if len(filter_dict.keys()) != 0:
            print("Filtering Events based on")
            pprint(filter_dict)
            for filter_field in filter_dict.keys():
                for event in eventList:
                    if event["eventHeader"][filter_field] == filter_dict[filter_field]:
                        eventListAnalysis.append(event)
        else:
            print("No Filter given - Analysis for all events")
            eventListAnalysis = eventList
        # Populate Metrics and Show preliminary results
        compressionRatioList = [Decimal(event["compressionRatio"]) for event in eventListAnalysis]
        timeTakenList = [Decimal(event["timeTaken"]) for event in eventListAnalysis]
        inputSizeList = [Decimal(event["inputSize"]) for event in eventListAnalysis]
        outputSizeList = [Decimal(event["outputSize"]) for event in eventListAnalysis]
        compressionSpeedList  = list(map(truediv,inputSizeList,timeTakenList))
        x_axis = range(len(eventListAnalysis))
        axscompressor.plot(x_axis,compressionSpeedList,label=compression_log["compressorConfig"])
                


##### Compression Ratio

In [ ]:
#Get a list of all the JSON files in the folder
import glob
jsonFiles = glob.glob(logsFolder + '/*.json')
# Get all the json files with the Compressor Key set to the target
targetFiles = []
for jsonfile in jsonFiles:
    with open(jsonfile) as f:
        jsonParse = json.load(f)
        if jsonParse["Compressor"]==Compressor:
            print (f'{f.name} ---> {jsonParse["Compressor"]}')
            targetFiles.append(jsonfile)
## Plotiing Metrics
plt.style.use('classic')
#figcompressor, axscompressor = plt.subplots(2,sharex=True)
figcompressor, axscompressor = plt.subplots()
figcompressor.suptitle("Metrics Display")
axscompressor.set_title("Compression Ratio")
#axscompressor[0].plot(x_axis,compressionSpeedList,'tab:orange')
# axscompressor[1].set_title("Compressin Ratio")
#axscompressor[1].plot(x_axis,compressionRatioList,'tab:purple')
for jsonFile in targetFiles:
    with open(jsonFile) as f:
        compression_log = json.load(f)
        eventList = compression_log["evdata"] # The event List
        # No We design a filter dictionary to take up events based on filters
        # Leave Empty if All Events are to be considered
        eventListAnalysis = []
        filter_dict = {
            #  "trig": "2"
        }
        if len(filter_dict.keys()) != 0:
            print("Filtering Events based on")
            pprint(filter_dict)
            for filter_field in filter_dict.keys():
                for event in eventList:
                    if event["eventHeader"][filter_field] == filter_dict[filter_field]:
                        eventListAnalysis.append(event)
        else:
            print("No Filter given - Analysis for all events")
            eventListAnalysis = eventList
        # Populate Metrics and Show preliminary results
        compressionRatioList = [Decimal(event["compressionRatio"]) for event in eventListAnalysis]
        timeTakenList = [Decimal(event["timeTaken"]) for event in eventListAnalysis]
        inputSizeList = [Decimal(event["inputSize"]) for event in eventListAnalysis]
        outputSizeList = [Decimal(event["outputSize"]) for event in eventListAnalysis]
        compressionSpeedList  = list(map(truediv,inputSizeList,timeTakenList))
        x_axis = range(len(eventListAnalysis))
        axscompressor.plot(x_axis,compressionRatioList,label=compression_log["compressorConfig"])

### Compression Speed and Compression Ratio - Interactive Graphs

#### Compression Speed

In [ ]:
#Get a list of all the JSON files in the folder
import glob
import plotly.graph_objects as go
jsonFiles = glob.glob(logsFolder + '/*.json')
# Get all the json files with the Compressor Key set to the target
targetFiles = []
fig = go.Figure()
for jsonfile in jsonFiles:
    with open(jsonfile) as f:
        jsonParse = json.load(f)
        if jsonParse["Compressor"]==Compressor:
            print (f'{f.name} ---> {jsonParse["Compressor"]}')
            targetFiles.append(jsonfile)
## Plotiing Metrics
for jsonFile in targetFiles:
    with open(jsonFile) as f:
        print("Now analyzing  ",f.name)
        compression_log = json.load(f)
        print("File for Compression  ",compression_log["Filename"])
        print("Compressor Config   :"+compression_log["compressorConfig"])
        eventList = compression_log["evdata"] # The event List
        # No We design a filter dictionary to take up events based on filters
        # Leave Empty if All Events are to be considered
        eventListAnalysis = []
        filter_dict = {
            #  "trig": "2"
        }
        if len(filter_dict.keys()) != 0:
            print("Filtering Events based on")
            pprint(filter_dict)
            for filter_field in filter_dict.keys():
                for event in eventList:
                    if event["eventHeader"][filter_field] == filter_dict[filter_field]:
                        eventListAnalysis.append(event)
        else:
            print("No Filter given - Analysis for all events")
            eventListAnalysis = eventList
        # Populate Metrics and Show preliminary results
        compressionRatioList = [Decimal(event["compressionRatio"]) for event in eventListAnalysis]
        timeTakenList = [Decimal(event["timeTaken"]) for event in eventListAnalysis]
        inputSizeList = [Decimal(event["inputSize"]) for event in eventListAnalysis]
        outputSizeList = [Decimal(event["outputSize"]) for event in eventListAnalysis]
        compressionSpeedList  = list(map(truediv,inputSizeList,timeTakenList))
        print("Average Compression Speed  ",statistics.mean(compressionSpeedList ))
        print("Standard Deviation in data  ",statistics.stdev(compressionSpeedList ))
        meanhere = statistics.mean(compressionSpeedList )
        stdevhere = statistics.stdev(compressionSpeedList )
        x_axis = range(len(eventListAnalysis))
        fig.add_trace(go.Scatter(
            x=list(x_axis),
            y=compressionSpeedList,
            mode='lines+markers',
            name=compression_log["compressorConfig"],
            hovertemplate =
            '<i>Compression Speed</i>:%{y:.3f}'+
            '<br>'+
            '<b>%{text}</b>',
            text = ['Event Tag: {} <br> Event Trigger: {}  <br> Mean: {} <br> StDev: {} '.format(event["eventHeader"]["tag"],event["eventHeader"]["trig"],meanhere,stdevhere) for event in eventListAnalysis],
        ))
        
fig.update_layout(
    title='Compression Speed in bytes per microsecond',
    xaxis_title='Event No',
    yaxis_title='Compression Speed',
    hoverlabel_align = 'right'
)

# Show the plot
fig.show()

#### Compression Ratio

In [ ]:
import glob
import plotly.graph_objects as go
fig2 = go.Figure()
jsonFiles = glob.glob(logsFolder + '/*.json')
# Get all the json files with the Compressor Key set to the target
targetFiles = []
fig = go.Figure()
for jsonfile in jsonFiles:
    with open(jsonfile) as f:
        jsonParse = json.load(f)
        if jsonParse["Compressor"]==Compressor:
            print (f'{f.name} ---> {jsonParse["Compressor"]}')
            targetFiles.append(jsonfile)
########################################################################################
for jsonFile in targetFiles:
    with open(jsonFile) as f:
        print("Now analyzing  ",f.name)
        compression_log = json.load(f)
        print("File for Compression  ",compression_log["Filename"])
        print("Compressor Config   :"+compression_log["compressorConfig"])
        eventList = compression_log["evdata"] # The event List
        # No We design a filter dictionary to take up events based on filters
        # Leave Empty if All Events are to be considered
        eventListAnalysis = []
        filter_dict = {
            #  "trig": "2"
        }
        if len(filter_dict.keys()) != 0:
            print("Filtering Events based on")
            pprint(filter_dict)
            for filter_field in filter_dict.keys():
                for event in eventList:
                    if event["eventHeader"][filter_field] == filter_dict[filter_field]:
                        eventListAnalysis.append(event)
        else:
            print("No Filter given - Analysis for all events")
            eventListAnalysis = eventList
        # Populate Metrics and Show preliminary results
        compressionRatioList = [Decimal(event["compressionRatio"]) for event in eventListAnalysis]
        print("Average Compression Ratio  ",statistics.mean(compressionRatioList))
        print("Standard Deviation in data  ",statistics.stdev(compressionRatioList))
        meanhere = statistics.mean(compressionRatioList)
        stdevhere = statistics.stdev(compressionRatioList)
        timeTakenList = [Decimal(event["timeTaken"]) for event in eventListAnalysis]
        inputSizeList = [Decimal(event["inputSize"]) for event in eventListAnalysis]
        outputSizeList = [Decimal(event["outputSize"]) for event in eventListAnalysis]
        compressionSpeedList  = list(map(truediv,inputSizeList,timeTakenList))
        x_axis = range(len(eventListAnalysis))
        #tagList = [event["eventHeader"]["tag"] for event in eventListAnalysis ]
        #trigList = [event["eventHeader"]["trig"] for event in eventListAnalysis ]
        fig2.add_trace(go.Scatter(
            x=list(x_axis),
            y=compressionRatioList,
            mode='lines+markers',
            name=compression_log["compressorConfig"],
            hovertemplate =
            '<i>Compression Ratio</i>:%{y:.3f}'+
            '<br>'+
            '<b>%{text}</b>',
            text = ['Event Tag: {} <br> Event Trigger: {}  <br> Mean: {} <br> StDev: {} '.format(event["eventHeader"]["tag"],event["eventHeader"]["trig"],meanhere,stdevhere) for event in eventListAnalysis],
                ))
fig2.update_layout(
    title='Compression Ratio',
    xaxis_title='Event No',
    yaxis_title='Compression Ratio',
    hoverlabel_align = 'right'
)

# Show the plot
fig2.show()
        

#### Compression Analysis with Input Load

##### Compression Speed

In [ ]:
import glob
import plotly.graph_objects as go
fig4 = go.Figure()
jsonFiles = glob.glob(logsFolder + '/*.json')
# Get all the json files with the Compressor Key set to the target
targetFiles = []
fig = go.Figure()
for jsonfile in jsonFiles:
    with open(jsonfile) as f:
        jsonParse = json.load(f)
        if jsonParse["Compressor"]==Compressor:
            print (f'{f.name} ---> {jsonParse["Compressor"]}')
            targetFiles.append(jsonfile)
########################################################################################
for jsonFile in targetFiles:
    with open(jsonFile) as f:
        print("Now analyzing  ",f.name)
        compression_log = json.load(f)
        print("File for Compression  ",compression_log["Filename"])
        print("Compressor Config   :"+compression_log["compressorConfig"])
        eventList = compression_log["evdata"] # The event List
        # No We design a filter dictionary to take up events based on filters
        # Leave Empty if All Events are to be considered
        eventListAnalysis = []
        filter_dict = {
            #  "trig": "2"
        }
        if len(filter_dict.keys()) != 0:
            print("Filtering Events based on")
            pprint(filter_dict)
            for filter_field in filter_dict.keys():
                for event in eventList:
                    if event["eventHeader"][filter_field] == filter_dict[filter_field]:
                        eventListAnalysis.append(event)
        else:
            print("No Filter given - Analysis for all events")
            eventListAnalysis = eventList
        # Populate Metrics and Show preliminary results
        compressionRatioList = [Decimal(event["compressionRatio"]) for event in eventListAnalysis]
        timeTakenList = [Decimal(event["timeTaken"]) for event in eventListAnalysis]
        inputSizeList = [Decimal(event["inputSize"]) for event in eventListAnalysis]
        outputSizeList = [Decimal(event["outputSize"]) for event in eventListAnalysis]
        compressionSpeedList  = list(map(truediv,inputSizeList,timeTakenList))
        compressionSpeedList  = list(map(truediv,inputSizeList,timeTakenList))
        print("Average Compression Speed  ",statistics.mean(compressionSpeedList ))
        print("Standard Deviation in data  ",statistics.stdev(compressionSpeedList ))
        meanhere = statistics.mean(compressionSpeedList)
        stdevhere = statistics.stdev(compressionSpeedList )
        x_axis = inputSizeList
        fig4.add_trace(go.Scatter(
            x=list(x_axis),
            y=compressionSpeedList,
            mode='markers',
            name=compression_log["compressorConfig"],
            hovertemplate =
            '<i>Compression Speed</i>:%{y:.3f}'+
            '<br><b>Input Size</b>: %{x:.1f}<br>'+
            '<br>'+
            '<b>%{text}</b>',
            text = ['Event Tag: {} <br> Event Trigger: {}  <br> Mean: {} <br> StDev: {} '.format(event["eventHeader"]["tag"],event["eventHeader"]["trig"],meanhere,stdevhere) for event in eventListAnalysis],
                ))
fig4.update_layout(
    title='Compression Speed',
    xaxis_title='Event Input Size',
    yaxis_title='Compression Speed',
    hoverlabel_align = 'right'
)

# Show the plot
fig4.show()

##### Compression Ratio

In [ ]:
import glob
import plotly.graph_objects as go
fig3 = go.Figure()
jsonFiles = glob.glob(logsFolder + '/*.json')
# Get all the json files with the Compressor Key set to the target
targetFiles = []
fig = go.Figure()
for jsonfile in jsonFiles:
    with open(jsonfile) as f:
        jsonParse = json.load(f)
        if jsonParse["Compressor"]==Compressor:
            print (f'{f.name} ---> {jsonParse["Compressor"]}')
            targetFiles.append(jsonfile)
########################################################################################
for jsonFile in targetFiles:
    with open(jsonFile) as f:
        print("Now analyzing  ",f.name)
        compression_log = json.load(f)
        print("File for Compression  ",compression_log["Filename"])
        print("Compressor Config   :"+compression_log["compressorConfig"])
        eventList = compression_log["evdata"] # The event List
        # No We design a filter dictionary to take up events based on filters
        # Leave Empty if All Events are to be considered
        eventListAnalysis = []
        filter_dict = {
            #  "trig": "2"
        }
        if len(filter_dict.keys()) != 0:
            print("Filtering Events based on")
            pprint(filter_dict)
            for filter_field in filter_dict.keys():
                for event in eventList:
                    if event["eventHeader"][filter_field] == filter_dict[filter_field]:
                        eventListAnalysis.append(event)
        else:
            print("No Filter given - Analysis for all events")
            eventListAnalysis = eventList
        # Populate Metrics and Show preliminary results
        compressionRatioList = [Decimal(event["compressionRatio"]) for event in eventListAnalysis]
        print("Average Compression Ratio  ",statistics.mean(compressionRatioList))
        print("Standard Deviation in data  ",statistics.stdev(compressionRatioList))
        meanhere = statistics.mean(compressionRatioList)
        stdevhere = statistics.stdev(compressionRatioList)
        timeTakenList = [Decimal(event["timeTaken"]) for event in eventListAnalysis]
        inputSizeList = [Decimal(event["inputSize"]) for event in eventListAnalysis]
        outputSizeList = [Decimal(event["outputSize"]) for event in eventListAnalysis]
        compressionSpeedList  = list(map(truediv,inputSizeList,timeTakenList))
        x_axis = inputSizeList
        #tagList = [event["eventHeader"]["tag"] for event in eventListAnalysis ]
        #trigList = [event["eventHeader"]["trig"] for event in eventListAnalysis ]
        fig3.add_trace(go.Scatter(
            x=list(x_axis),
            y=compressionRatioList,
            mode='markers',
            name=compression_log["compressorConfig"],
            hovertemplate =
            '<i>Compression Ratio</i>:%{y:.3f}'+
            '<br><b>Input Size</b>: %{x:.1f}<br>'+
            '<br>'+
            '<b>%{text}</b>',
            text = ['Event Tag: {} <br> Event Trigger: {}  <br> Mean: {} <br> StDev: {} '.format(event["eventHeader"]["tag"],event["eventHeader"]["trig"],meanhere,stdevhere) for event in eventListAnalysis],
                ))
fig3.update_layout(
    title='Compression Ratio',
    xaxis_title='Event Input Size',
    yaxis_title='Compression Ratio',
    hoverlabel_align = 'right'
)

# Show the plot
fig3.show()

### Decompression Speed

##### Decompression Speed Over Events

In [ ]:
import glob
jsonFiles = glob.glob(logsFolder + '/*.json')
# Get all the json files with the Compressor Key set to the target
targetFiles = []
for jsonfile in jsonFiles:
    with open(jsonfile) as f:
        jsonParse = json.load(f)
        if jsonParse["Compressor"]==Compressor+":: Decompression Run ":
            targetFiles.append(jsonfile)
## Plotiing Metrics
plt.style.use('classic')
#figcompressor, axscompressor = plt.subplots(2,sharex=True)
figcompressor, axscompressor = plt.subplots()
figcompressor.suptitle("Metrics Display")
axscompressor.set_title("Decompression Speed in bytes per microsecond")
#axscompressor[0].plot(x_axis,compressionSpeedList,'tab:orange')
# axscompressor[1].set_title("Compressin Ratio")
#axscompressor[1].plot(x_axis,compressionRatioList,'tab:purple')
for jsonFile in targetFiles:
    with open(jsonFile) as f:
        compression_log = json.load(f)
        eventList = compression_log["evdata"] # The event List
        # No We design a filter dictionary to take up events based on filters
        # Leave Empty if All Events are to be considered
        eventListAnalysis = []
        filter_dict = {
            #  "trig": "2"
        }
        if len(filter_dict.keys()) != 0:
            print("Filtering Events based on")
            pprint(filter_dict)
            for filter_field in filter_dict.keys():
                for event in eventList:
                    if event["eventHeader"][filter_field] == filter_dict[filter_field]:
                        eventListAnalysis.append(event)
        else:
            print("No Filter given - Analysis for all events")
            eventListAnalysis = eventList
        # Populate Metrics and Show preliminary results
        compressionRatioList = [Decimal(event["compressionRatio"]) for event in eventListAnalysis]
        timeTakenList = [Decimal(event["timeTaken"]) for event in eventListAnalysis]
        inputSizeList = [Decimal(event["inputSize"]) for event in eventListAnalysis]
        outputSizeList = [Decimal(event["outputSize"]) for event in eventListAnalysis]
        decompressionSpeedList  = list(map(truediv,outputSizeList,timeTakenList))
        x_axis = range(len(eventListAnalysis))
        axscompressor.plot(x_axis,decompressionSpeedList,label=compression_log["compressorConfig"])

#### Decompression Speed Interactive Graphs

##### Decompression Speed over events

In [ ]:

#Get a list of all the JSON files in the folder
import glob
import plotly.graph_objects as go
jsonFiles = glob.glob(logsFolder + '/*.json')
# Get all the json files with the Compressor Key set to the target
targetFiles = []
fig5 = go.Figure()
fig6 = go.Figure()
for jsonfile in jsonFiles:
    with open(jsonfile) as f:
        jsonParse = json.load(f)
        if jsonParse["Compressor"]==Compressor+":: Decompression Run ":
            print (f'{f.name} ---> {jsonParse["Compressor"]}')
            targetFiles.append(jsonfile)
## Plotiing Metrics
for jsonFile in targetFiles:
    with open(jsonFile) as f:
        print("Now analyzing  ",f.name)
        compression_log = json.load(f)
        print("File for Compression  ",compression_log["Filename"])
        print("Compressor Config   :"+compression_log["compressorConfig"])
        eventList = compression_log["evdata"] # The event List
        # No We design a filter dictionary to take up events based on filters
        # Leave Empty if All Events are to be considered
        eventListAnalysis = []
        filter_dict = {
            #  "trig": "2"
        }
        if len(filter_dict.keys()) != 0:
            print("Filtering Events based on")
            pprint(filter_dict)
            for filter_field in filter_dict.keys():
                for event in eventList:
                    if event["eventHeader"][filter_field] == filter_dict[filter_field]:
                        eventListAnalysis.append(event)
        else:
            print("No Filter given - Analysis for all events")
            eventListAnalysis = eventList
        # Populate Metrics and Show preliminary results
        compressionRatioList = [Decimal(event["compressionRatio"]) for event in eventListAnalysis]
        timeTakenList = [Decimal(event["timeTaken"]) for event in eventListAnalysis]
        inputSizeList = [Decimal(event["inputSize"]) for event in eventListAnalysis]
        outputSizeList = [Decimal(event["outputSize"]) for event in eventListAnalysis]
        decompressionSpeedList  = list(map(truediv,outputSizeList,timeTakenList))
        meanhere = statistics.mean(decompressionSpeedList)
        stdevhere = statistics.stdev(decompressionSpeedList)
        print("Average Compression Speed  ",statistics.mean(decompressionSpeedList))
        print("Standard Deviation in data  ",statistics.stdev(decompressionSpeedList))
        x_axis = range(len(eventListAnalysis))
        fig5.add_trace(go.Scatter(
            x=list(x_axis),
            y=decompressionSpeedList,
            mode='lines+markers',
            name=compression_log["compressorConfig"],
            hovertemplate =
            '<i>Decompression Ratio</i>:%{y:.3f}'+
            '<br>'+
            '<b>%{text}</b>',
            text = ['Event Tag: {} <br> Event Trigger: {}  <br> Mean: {} <br> StDev: {} '.format(event["eventHeader"]["tag"],event["eventHeader"]["trig"],meanhere,stdevhere) for event in eventListAnalysis],
        ))
        fig6.add_trace(go.Scatter(
            x=list(outputSizeList),
            y=decompressionSpeedList,
            mode='markers',
            name=compression_log["compressorConfig"],
            hovertemplate =
            '<i>Decompression Speed</i>:%{y:.3f}'+
            '<br>'+
            '<b>%{text}</b>',
            text = ['Event Tag: {} <br> Event Trigger: {}  <br> Mean: {} <br> StDev: {} '.format(event["eventHeader"]["tag"],event["eventHeader"]["trig"],meanhere,stdevhere) for event in eventListAnalysis],
        ))
        
fig5.update_layout(
    title='Decompression Speed in bytes per microsecond',
    xaxis_title='Event No',
    yaxis_title='Decompression Speed',
    hoverlabel_align = 'right'
)
fig6.update_layout(
    title='Decompression Speed in bytes per microsecond',
    xaxis_title='Event Output Size',
    yaxis_title='Decompression Speed',
    hoverlabel_align = 'right'
)

# Show the plot
fig5.show()

##### Output Load vs Decompression speed 

In [ ]:
fig6.show()

## Closing and Cleaning Up

In [ ]:
compressionLogFile.close()
decompressionLogFile.close()